# Алгоритмы интеллектуальной обработки больших объемов данных
## Семинар, учимся настраивать параметры бустинга


Разберем, например, xgboost:
https://xgboost.readthedocs.io/en/latest/parameter.html

https://xgboost.readthedocs.io/en/latest/python/python_api.html

Параметры можно разделить на группы. 



1) Общие параметры алгоритма, например, тип бустинга

2) Параметры обучения бустинга, например число итераций, learning rate

3) Параметры построения деревьев, например, глубина, минимальное число элементов в листе

4) Параметры стохастики, subsample, colsample

5) Регуляриазция, например, lambda


ПРАВИЛЬНОГО алгоритма для тюнинга нет, вот один из вариантов, которые часто используют на практике.

1) Возьмите дефолтное параметры. Поставьте нужные loss и eval функции.  Измените максимальное  число деревьев, чтобы было не долго делать эксперименты

2) Подберите для него alpha, чтобы алгоритм не переобучался

3) Подберите параметры сложности дерева, например,  глубину, мин число объектов в листах, мин значение для расщепления и тд

4) Подберите паарметры стохастики

5) Подберите регуляризацию

6) Уменьшите leraning rate и обучите максимальное число деревьев, чтобы не переобучалось

Давайте настроим параметры бустинга на нашем конкурсе!

In [1]:
doc_to_title = {}
with open('docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))


28026


In [2]:
import pandas as pd
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [3]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:15]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 15) (11690,) (11690,)


Подберите размер батча для обучения. Линейная модель не должна учиться дольше нескольких минут. 

Не забывайте использовать скейлер!

In [4]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
#clf = xgb.XGBClassifier(use_label_encoder = False, eval_metric = 'logloss',)
#clf = lgb.LGBMClassifier()

In [5]:
scaler = StandardScaler()
scaler.fit(X_train)
train_scale = scaler.transform(X_train)
dtrain = xgb.DMatrix(train_scale, label=y_train)

In [7]:
params = {
    'verbosity' :'0', 'use_label_encoder' : False, 'n_estimators' : 800,
    'objective' : 'binary:hinge' , 'reg_lambda' : 0.018, 'max_depth' : 6,
    'gamma': 0.01, 'min_child_weight': 1, 'subsample' : 1, 
    'max_delta_step' : 1, 'base_score' : 4, 'learning_rate' : 0.01
}

In [8]:
# Вот тут делался перебор гиперпараметров, лучшие результаты вносились в params
temp = [0.01, 0.1, 0.001]
for i in temp:
    clf = xgb.XGBClassifier(verbosity = 0, use_label_encoder = False, n_estimators = 500,
                           objective = 'binary:hinge' , reg_lambda = 0.018, max_depth = 6,
                           gamma = i, min_child_weight = 1, subsample = 0.8, 
                           max_delta_step = 1, base_score = 4, learning_rate = 0.1)
    scores = cross_val_score(clf, train_scale, y_train, cv = 7)
    score = scores.mean()
    print(score)


0.7832335329341318
0.7839178785286568
0.7828058169375532


Задача: обучите на этих фичах (число фичей рассатриваем как гипераматр) бустинг, подберите параметры, засабмитьте на кагл. Побейте 0.62 паблик скор за сутки.  Удачи!

In [9]:
test_data = pd.read_csv('test_groups.csv')
X_train.shape, y_train.shape,train_data.shape, test_data.shape

((11690, 15), (11690,), (11690, 4), (16627, 3))

In [10]:
traingroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    #target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title,))

In [11]:
X_test = []
groups_test = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:15]    )
X_test = np.array(X_test)
groups_test = np.array(groups_test)
print (X_test.shape, groups_test.shape)

(16627, 15) (16627,)


In [13]:
 clf = xgb.XGBClassifier(verbosity = 0, use_label_encoder = False, n_estimators = 500,
                           objective = 'binary:hinge' , reg_lambda = 0.018, max_depth = 6,
                           gamma = 0.01, min_child_weight = 1, subsample = 1, 
                           max_delta_step = 1, base_score = 4, learning_rate = 0.1)
clf.fit(train_scale, y_train)
#clf = xgb.train(params, dtrain, num_boost_round = 10)

XGBClassifier(base_score=4, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.01, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=1, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=8, num_parallel_tree=1,
              objective='binary:hinge', random_state=0, reg_alpha=0,
              reg_lambda=0.018, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=0)

In [14]:
X_scaled = scaler.transform(X_test)
#dtest = xgb.DMatrix(X_scaled)
pred = clf.predict(X_scaled)
scores = cross_val_score(clf, train_scale, y_train, cv = 5)
scores.mean()

0.7781009409751926

In [15]:
ans1 = np.array(test_data['pair_id']).reshape(-1,1)
ans = np.append(ans1, pred.reshape(-1,1), axis = 1)
title = ['pair_id', 'target']
df = pd.DataFrame(ans, columns = title)
df.to_csv("submission.csv", index=False)

In [1]:
# Ник на kaggle: v nebe lunka (Илья Масс)